In [45]:
import torch 
import torch.nn as nn
import fasttext as ft
import math

In [ ]:
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz -P {path}
! wget -c https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz
# !gunzip cc.en.300.bin cc.en.300.bin.gz
!gunzip cc.fr.300.bin cc.fr.300.bin.gz

In [ ]:
en = ft.load_model(str('cc.en.300.bin'))
fr = ft.load_model(str('cc.fr.300.bin'))

# New Implementation

In [21]:
d_model = 5

In [25]:
def getWordVectors(sentence):
    sentence = sentence.split(' ')
    vecs = torch.Tensor(len(sentence),d_model)
    return vecs

In [51]:
def PositionalEncoding(wordVecs):
    for pos in range(wordVecs.shape[0]):
        for i in range(wordVecs[pos].shape[0]):
            if i%2 == 0:
                wordVecs[pos][i] = wordVecs[pos][i] + math.sin(pos/(10000**(2*i/d_model)))
            else:
                wordVecs[pos][i] = wordVecs[pos][i] + math.cos(pos/(10000**(2*i/d_model)))
                
    return wordVecs
                                           

In [57]:
def get_qkv_weights(r,c):
    query_weights = torch.randn(r,c)
    key_weights = torch.randn(r,c)
    value_weights = torch.randn(r,c)
    return query_weights, key_weights, value_weights
    

In [69]:
def qkvs(vectorMatrix, new_dim):
    
    query_weights, key_weights, value_weights = get_qkv_weights(5,new_dim)
    
    return torch.matmul(vectorMatrix, query_weights), torch.matmul(vectorMatrix, key_weights), \
    torch.matmul(vectorMatrix, value_weights) 

In [ ]:
'''
wordVecs = getWordVectors('Hi there this is nuts')
pos_encoded = PositionalEncoding(wordVecs)

new_dim = 3
queries, keys, values = qkvs(pos_encoded, new_dim)



'''

In [ ]:
wordVecs = getWordVectors('Hi there this is nuts')
wordVecs

In [ ]:
pos_encoded = PositionalEncoding(wordVecs)
pos_encoded

In [70]:
new_dim = 3
queries, keys, values = qkvs(pos_encoded, new_dim)